In [ ]:
import torchaudio

data_dir = "data"
trainset_speechcommands = torchaudio.datasets.SPEECHCOMMANDS(
        f"./{data_dir}/",
        download=True,
        subset="training",
    )
testset_speechcommands = torchaudio.datasets.SPEECHCOMMANDS(
    f"./{data_dir}/",
    download=True,
    subset="testing",
)
evalset_speechcommands = torchaudio.datasets.SPEECHCOMMANDS(
    f"./{data_dir}/", download=True, subset="validation"
)

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from os.path import join
import torch

dataset_name = "speechcommand"
config = {
    "order": "03",
    "task": "multiclass",
    "data": {
        "batch_size": 32,
        "size": 8000,
    },
    "network": {
        "input_channels": 1,
        "linear_size": 31680,
        "num_classes": 35,
        "first_layer_kernel_size": 80,
        "second_layer_kernel_size": 3,
    },
    "train": {
        "criterion": torch.nn.CrossEntropyLoss(),
        "checkpoint_save_dir": "checkpoints",
        "epochs": 100,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
    },
}

In [ ]:
ORDER = config["order"]
SIZE = config["data"]["size"]
TASK = config["task"]
NUM_CLASSES = config["network"]["num_classes"]
EPOCHS = config["train"]["epochs"]
BATCH_SIZE = config["data"]["batch_size"]
INPUT_CHANNEL = config["network"]["input_channels"]
LINEAR_SIZE = config["network"]["linear_size"]
CRITERION = config["train"]["criterion"]
DEVICE = config["train"]["device"]
FIRST_LAYER_KERNEL_SIZE = config["network"]["first_layer_kernel_size"]
SECOND_LAYER_LAYER_KERNEL_SIZE = config["network"]["second_layer_kernel_size"]

experiment_name = f"{ORDER}-{dataset_name}-{SIZE}"

In [ ]:
checkpoint_save_dir = join(config["train"]["checkpoint_save_dir"], experiment_name)

In [ ]:
from tcnn.utils.simulation.dataset.load_datasets import load_speech_command

(
    train_loader,
    test_loader,
    eval_dataloader,
    input_shape,
    output_shape,
    labels,
) = load_speech_command(BATCH_SIZE, "data")

In [ ]:
import lenet_models

model_dict = lenet_models.get_constant_model_dict(
    INPUT_CHANNEL,
    NUM_CLASSES,
    LINEAR_SIZE,
    FIRST_LAYER_KERNEL_SIZE,
    SECOND_LAYER_LAYER_KERNEL_SIZE,
)

In [ ]:
def get_optimizers_dict(model_dict):
    """
    Get a dictionary of optimizers for the models.
    """
    return {
        model_name: torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
        for model_name, model in model_dict.items()
    }


def get_scheduler_dict(optimizer_dict):
    """
    Get a dictionary of schedulers for the optimizers.
    """
    return {
        model_name: torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        for model_name, optimizer in optimizer_dict.items()
    }

In [ ]:
optimizers_dict = get_optimizers_dict(model_dict)
schedulers_dict = get_scheduler_dict(optimizers_dict)

In [ ]:
history_dict = {}

In [ ]:
from tcnn.utils.experiment.train import train_and_test_model

for model_name, model in model_dict.items():
    print(f"Training model {model_name}")
    history_dict[model_name] = train_and_test_model(
        model,
        train_loader,
        test_loader,
        CRITERION,
        optimizers_dict[model_name],
        scheduler=schedulers_dict[model_name],
        epochs=EPOCHS,
        save_checkpoint=True,
        save_checkpoint_interval=1,
        checkpoint_save_dir=join(checkpoint_save_dir, model_name),
        task=TASK,
    )
    print("***" * 10)

In [ ]:
from tcnn.utils.experiment.plot import plot_history

for model_name, history in history_dict.items():
    print(f"Model {model_name} history:")
    plot_history(history, model_name)

In [ ]:
import torch

input_shape = (BATCH_SIZE, 1, SIZE)
input_tensor = torch.randn(input_shape).to(DEVICE)

In [ ]:
import torchprofile
from tcnn.utils.experiment.model import count_parameters
from tcnn.utils.experiment.train import eval_model

result = dict()
for model_name, model in model_dict.items():
    print(f"Evaluating model {model_name}")
    result[model_name] = dict()
    result[model_name]["macs"] = torchprofile.profile_macs(model, input_tensor)
    result[model_name]["params"] = count_parameters(model)
    result[model_name]["performance"] = eval_model(model, test_loader, CRITERION, TASK)
    with torch.no_grad():
        torch.cuda.empty_cache()
    print("***" * 10)

In [ ]:
from tcnn.utils.experiment.plot import plot_history_dict

plot_history_dict(history_dict)

In [ ]:
from tcnn.utils.experiment.log import show_test_result

show_test_result(result)